In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

base_path = "input/"
test_dat = pd.read_csv(base_path + 'test.csv')
train_dat = pd.read_csv(base_path + 'train.csv')
submission = pd.read_csv(base_path + 'sample_submission.csv')

train_y = train_dat['target']
train_x = train_dat.drop(['target', 'id'], axis = 1)
test_dat = test_dat.drop(['id'], axis = 1)

merged_dat = pd.concat([train_x, test_dat],axis=0)

#change data to float32
for c, dtype in zip(merged_dat.columns, merged_dat.dtypes): 
    if dtype == np.float64:     
        merged_dat[c] = merged_dat[c].astype(np.float32)

#one hot encode the categoricals
cat_features = [col for col in merged_dat.columns if col.endswith('cat')]
for column in cat_features:
    temp=pd.get_dummies(pd.Series(merged_dat[column]))
    merged_dat=pd.concat([merged_dat,temp],axis=1)
    merged_dat=merged_dat.drop([column],axis=1)

#standardize the scale of the numericals
numeric_features = [col for col in merged_dat.columns if '_calc_' in  str(col)]
numeric_features = [col for col in numeric_features if '_bin' not in str(col)]

scaler = StandardScaler()
scaled_numerics = scaler.fit_transform(merged_dat[numeric_features])
scaled_num_df = pd.DataFrame(scaled_numerics, columns =numeric_features )


merged_dat = merged_dat.drop(numeric_features, axis=1)

merged_dat = np.concatenate((merged_dat.values,scaled_num_df), axis = 1)

train_x = merged_dat[:train_x.shape[0]]
test_dat = merged_dat[train_x.shape[0]:]


config = tf.contrib.learn.RunConfig(tf_random_seed=42)

feature_cols = tf.contrib.learn.infer_real_valued_columns_from_input(train_x)

dnn_clf = tf.contrib.learn.DNNClassifier(hidden_units=[150,150,150], n_classes=2,
                                         feature_columns=feature_cols, config=config)

dnn_clf.fit(train_x, train_y, batch_size=50, steps=40000)

INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_master': '', '_model_dir': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_save_checkpoints_steps': None, '_is_chief': True, '_evaluation_master': '', '_save_summary_steps': 100, '_environment': 'local', '_keep_checkpoint_every_n_hours': 10000, '_task_type': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002E459A6A630>, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_tf_random_seed': 42, '_task_id': 0, '_num_worker_replicas': 0}
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Argume

DNNClassifier(params={'input_layer_min_slice_size': None, 'optimizer': None, 'hidden_units': [150, 150, 150], 'gradient_clip_norm': None, 'dropout': None, 'head': <tensorflow.contrib.learn.python.learn.estimators.head._BinaryLogisticHead object at 0x000002E45FD149B0>, 'feature_columns': (_RealValuedColumn(column_name='', dimension=227, default_value=None, dtype=tf.float64, normalizer=None),), 'embedding_lr_multipliers': None, 'activation_fn': <function relu at 0x000002E45DBB92F0>})

In [4]:
dnn_y_pred = dnn_clf.predict_proba(test_dat)


dnn_out = list(dnn_y_pred)

dnn_output = submission
dnn_output['target'] = [x[1] for x in dnn_out]


dnn_output.to_csv(base_path + 'test_dnn_predictions.csv', index=False, float_format='%.4f')

Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
INFO:tensorflow:Restoring parameters from C:\Users\CROSSB~1\AppData\Local\Temp\tmpl9f8ngxl\model.ckpt-40000


ResourceExhaustedError: OOM when allocating tensor with shape[892816,227]
	 [[Node: ToFloat = Cast[DstT=DT_FLOAT, SrcT=DT_DOUBLE, _device="/job:localhost/replica:0/task:0/gpu:0"](_recv_input_0/_81)]]
	 [[Node: dnn/binary_logistic_head/predictions/probabilities/_83 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_19_dnn/binary_logistic_head/predictions/probabilities", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'ToFloat', defined at:
  File "C:\Users\Crossbell\Anaconda2\envs\tensorflow\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Crossbell\Anaconda2\envs\tensorflow\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Crossbell\Anaconda2\envs\tensorflow\lib\site-packages\ipykernel\__main__.py", line 3, in <module>
    app.launch_new_inst

In [ ]:
dnn_y_pred_train = dnn_clf.predict_proba(train_x)
train = pd.read_csv(base_path + 'train.csv')
sub = pd.DataFrame()
sub['id'] = train['id']
sub['target'] = [x[1] for x in list(dnn_y_pred_train)]
sub.to_csv(base_path + 'train_dnn_predictions.csv', index=False, float_format='%.4f')